#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 2:  PageRank + Learning to Rank

### 100 points [10% of your final grade]

### Due: March 5, 2020 by 11:59pm

*Goals of this homework:* In this homework you will explore real-world challenges of building a graph (in this case, from tweets), implement and test the classic PageRank algortihm over this graph. In addition, you will apply learning to rank to a real-world dataset and report the performance in terms of NDCG.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: PageRank (60 points)
In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. 


## Part 1.1: A re-Tweet Graph (20 points)

So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is retweeted by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *PageRank.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are binary and directed. If Bob retweets Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
# Here define your function for building the graph by parsing 
# the input file of tweets
# Insert as many cells as you want

In [2]:
# Call your function to print out the size of the graph, 
# i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you 
#write the graph into a file, and load it later.

In [9]:
import json
from collections import defaultdict

edges = 0
graph = defaultdict(dict)


def addEdge(userID, sourceID):
    if sourceID in graph[userID]:
        graph[userID][sourceID] = 1
    else:
        graph[userID][sourceID] = 1
    if sourceID not in graph:
        graph[sourceID][userID] = 0 
        

data_lines = open('HITS.json', encoding='UTF-8')
for line in data_lines:
    data = json.loads(line)
    userID = data['user']['id']
    sourceID = data['retweeted_status']['user']['id']
    if userID == sourceID:
        continue
    addEdge(userID, sourceID)
data_lines.close()
    
    
def key_id(keys_sorted):
    keys_id_dict = {}
    j = 0
    for i in keys_sorted:
        keys_id_dict[i] = j
        j += 1
    return keys_id_dict


# graph -> matrix
def get_matrix(graph):
    keys_sorted = sorted(graph.keys())
    size = len(keys_sorted)
    M = [[0] * size for i in range(size)]
    keys_id_dict = key_id(keys_sorted)
    for k1 in keys_sorted:
        for k2 in keys_sorted:
            if k1 == k2:
                M[keys_id_dict[k1]][keys_id_dict[k2]] = 0
            try:
                M[keys_id_dict[k1]][keys_id_dict[k2]] = graph[k1][k2]
            except:
                M[keys_id_dict[k1]][keys_id_dict[k2]] = 0

    return M, keys_id_dict


M, keys_id_dict = get_matrix(graph)


import numpy as np
b = np.array(M,dtype = float)
print("The number of nodes: ", b.shape[0])
# print(b.shape[1])


count = 0
for i in range(b.shape[0]):
    for j in range(b.shape[1]):
        if M[i][j] == 1:
            count += 1
print("The number of edges: ", count)

The number of nodes:  1003
The number of edges:  6177


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## Part 1.2: PageRank Implementation (30 points)

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never retweeted and does not retweet anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

*ADD YOUR ANSWER HERE*

In [3]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want

# def PageRanker(...):
#    ...


In [10]:
a = np.transpose(b)


def graphMove(a):
    c = np.zeros((1003, 1003),dtype = float)
    for i in range(1003):
        for j in range(1003):
            if b[j].sum()==0:
                c[i][j] = 0
            else:
                c[i][j] = a[i][j] / (b[j].sum())
    return c


def firstPr():
    pr = np.zeros((1003, 1), dtype=float)
    for i in range(1003):
        pr[i] = float(1)/1003
    return pr


mm = np.zeros((1003, 1003), dtype=float)
for i in range (1003):
    for j in range (1003):
        mm[i][j] = 1/1003
        
def PageRanker(p,m,pr):
    T = p * m + (1 - p) * mm
    pr = np.transpose(pr)
    for i in range(100):
        pr = np.dot(pr, T)
        pr /= np.sum(pr)
    return pr

M = graphMove(a)
M = np.transpose(M)
pr = firstPr()
p = 0.9

In [4]:
# Now let's call your function on the graph you've built. Output the results.

In [11]:
PR = PageRanker(p,M,pr)


# print
def key_id(keys_sorted):
    keys_id_dict = {}
    j = 0
    for i in keys_sorted:
        keys_id_dict[i] = j
        j += 1
    return keys_id_dict


pr_dict = {}
for i in range(1003):
    pr_dict[i] = PR[0][i]

    
values = sorted(pr_dict.items(), key=lambda d: d[1], reverse=True)  


for i in range(10):
    for k,v in keys_id_dict.items():
        if v == values[i][0]:
            print(k, '\t', values[i][1])

1183906148 	 0.03690928484050658
2598548166 	 0.023700823719667377
3019659587 	 0.022420060162606687
3077695572 	 0.022046237452155906
3154266823 	 0.01990739513690696
3042570996 	 0.01977500924476432
3068694151 	 0.01954808142465658
3264645911 	 0.017964127148931492
3082766914 	 0.017457986143068278
571198546 	 0.016351985674481474


## Part 1.3: Improving PageRank (10 points)
In the many years since PageRank was introduced, there have been many improvements and extensions. For this part, you should experiment with one such improvement and then compare the results you get with the original results in Part 1.2. 

In [5]:
# Here add your code

In [12]:
graph = defaultdict(dict)


def addEdge(userID, sourceID):
    if sourceID in graph[userID]:
        graph[userID][sourceID] += 1
    else:
        graph[userID][sourceID] = 1
    if sourceID not in graph:
        graph[sourceID][userID] = 0
        

data_lines = open('HITS.json', encoding='UTF-8')
for line in data_lines:
    data = json.loads(line)
    userID = data['user']['id']
    sourceID = data['retweeted_status']['user']['id']
    if userID == sourceID:
        continue
    addEdge(userID, sourceID)
data_lines.close()
    
    
M, keys_id_dict = get_matrix(graph)


b = np.array(M,dtype = float)
a = np.transpose(b)
M = graphMove(a)
M = np.transpose(M)
pr = firstPr()
p = 0.9


PR = PageRanker(p,M,pr)


pr_dict = {}
for i in range(1003):
    pr_dict[i] = PR[0][i]

    
values = sorted(pr_dict.items(), key=lambda d: d[1], reverse=True)  
   
    
for i in range(10):
    for k,v in keys_id_dict.items():
        if v == values[i][0]:
            print(k, '\t', values[i][1])

3042570996 	 0.058613667318172254
2860872854 	 0.04766331262265769
1183906148 	 0.0338283838786767
3142161801 	 0.026258939442898775
610166901 	 0.025730931339386323
3154266823 	 0.02567377376546478
2598548166 	 0.023083933063185732
3198584744 	 0.021613601439217874
3156878078 	 0.020016610074063287
3169039209 	 0.019856262064168316


In [6]:
# Plus be sure to describe your extension (what is it? 
The edges are not binary any more. Instead, the value of the edge in matrix is the number of edge from userID to sourceID.
# why did you choose it?) and your comparison to Part 1.2
Because if A retweets B more times, it means B is of more value or importance.
So by this method, the top10 results are more reasonable than Part 1.2

# Part 2: Learning to Rank (40 points)

For this part, we're going to play with some Microsoft LETOR data that has query-document relevance judgments. Let's see how learning to rank works in practice. 

First, you will need to download the MQ2008.zip file from the Resources tab on Piazza. This is data from the [Microsoft Research IR Group](https://www.microsoft.com/en-us/research/project/letor-learning-rank-information-retrieval/).

The data includes 15,211 rows. Each row is a query-document pair. The first column is a relevance label of this pair (0,1 or 2--> the higher value the more related), the second column is query id, the following columns are features, and the end of the row is comment about the pair, including id of the document. A query-document pair is represented by a 46-dimensional feature vector. Features are a numeric value describing a document and query such as TFIDF, BM25, Page Rank, .... You can find compelete description of features from [here](https://arxiv.org/ftp/arxiv/papers/1306/1306.2597.pdf).

The good news for you is the dataset is ready for analysis: It has already been split into 5 folds (see the five folders called Fold1, ..., Fold5).


## Part 2.1: Build Point-wise Learning to Rank  (20 points)
First, you should build a point-wise Learning to Rank framework. 
1. You could train a binary classification model like SVM or logistic regression on the train file. In this case, 0 is treated as negative (irrelevant) sample and 1, 2 are treated as positive (relevant) sample.
2. You apply the already trained model to predict the scores for documents on test file.
3. Order the documents based on the scores.

add your results and discussion here

In [13]:
def createTrainDataSet(path):
    f = open(path, encoding='UTF-8')
    line = f.readline()
    x_train = []
    y_train = []
    while line:
        each_line_list = []
        j = 2
        line_list = line.split()
        for i in range(46):
            each_line_list.append(float(line_list[j].split(":")[1]))
            j = j + 1
        x_train.append(each_line_list)
        y_train.append(int(line_list[0]))
        line = f.readline()
    f.close()
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    for i in range(len(y_train)):
        if y_train[i] != 0:
            y_train[i] = 1
    return x_train, y_train
            
            
def createValiDataSet(path):
    f = open(path, encoding='UTF-8')
    line = f.readline()
    x_vali = []
    y_vali = []
    while line:
        each_line_list = []
        j = 2
        line_list = line.split()
        for i in range(46):
            each_line_list.append(float(line_list[j].split(":")[1]))
            j = j + 1
        x_vali.append(each_line_list)
        y_vali.append(int(line_list[0]))
        line = f.readline()
    f.close()
    x_vali = np.array(x_vali)
    y_vali = np.array(y_vali)
    for i in range(len(y_vali)):
        if y_vali[i] != 0:
            y_vali[i] = 1
    return x_vali, y_vali
    

def createTestDataSet(path):
    f = open(path, encoding='UTF-8')
    line = f.readline()
    x_test = []
    y_test = []
    qid_test = []
    docid_test = []
    rel_test = []
    while line:
        each_line_list = []
        j = 2
        line_list = line.split()
        for i in range(46):
            each_line_list.append(float(line_list[j].split(":")[1]))
            j = j + 1
        x_test.append(each_line_list)
        y_test.append(int(line_list[0]))
        rel_test.append(int(line_list[0]))
        qid_test.append(int(line_list[1].split(":")[1]))
        docid_test.append(line_list[50].split()[0])
        line = f.readline()
    f.close()
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    rel_test = np.array(rel_test)
    # qid_test = np.array(qid_test)
    for i in range(len(y_test)):
        if y_test[i] != 0:
            y_test[i] = 1
    return x_test, y_test, rel_test, qid_test, docid_test

In [14]:
def sorted_docid_print():
    # prepare for print
    c = clf.predict_log_proba (x_test)
    
    score_list = []
    for i in range(len(c)):
        score_list.append(c[i][1])
    
    qid_docid_score_dict = {}
    
    for i in range(len(c)):
        qid_docid_list = [docid_test[i], score_list[i]]
        if qid_test[i] not in qid_docid_score_dict:
            qid_docid_score_dict[qid_test[i]] = [qid_docid_list]
        else:
            qid_docid_score_dict[qid_test[i]].append(qid_docid_list)

    # print
    for key, value in qid_docid_score_dict.items():
        value.sort(key=lambda x: x[1], reverse=True)
        print("qid: ", key)
        for i in range(len(value)):
            print(value[i][0], '\t', value[i][1])

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold1\\train.txt")
x_vali, y_vali = createValiDataSet("MQ2008\\MQ2008\\Fold1\\vali.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold1\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
predictions = clf.predict(x_vali)
print("Accuracy on {} vali file is {}".format("Fold1", metrics.accuracy_score(y_vali, predictions)))
# apply the model on test file and print
print("For each qid in Fold1 test file, sort the documents:")
sorted_docid_print()

In [ ]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold2\\train.txt")
x_vali, y_vali = createValiDataSet("MQ2008\\MQ2008\\Fold2\\vali.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold2\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
predictions = clf.predict(x_vali)
print("Accuracy on {} vali file is {}".format("Fold2", metrics.accuracy_score(y_vali, predictions)))
# apply the model on test file and print
print("For each qid in Fold2 test file, sort the documents:")
sorted_docid_print()

In [ ]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold3\\train.txt")
x_vali, y_vali = createValiDataSet("MQ2008\\MQ2008\\Fold3\\vali.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold3\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
predictions = clf.predict(x_vali)
print("Accuracy on {} vali file is {}".format("Fold3", metrics.accuracy_score(y_vali, predictions)))
# apply the model on test file and print
print("For each qid in Fold3 test file, sort the documents:")
sorted_docid_print()

In [ ]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold4\\train.txt")
x_vali, y_vali = createValiDataSet("MQ2008\\MQ2008\\Fold4\\vali.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold4\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
predictions = clf.predict(x_vali)
print("Accuracy on {} vali file is {}".format("Fold4", metrics.accuracy_score(y_vali, predictions)))
# apply the model on test file and print
print("For each qid in Fold4 test file, sort the documents:")
sorted_docid_print()

In [ ]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold5\\train.txt")
x_vali, y_vali = createValiDataSet("MQ2008\\MQ2008\\Fold5\\vali.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold5\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
predictions = clf.predict(x_vali)
print("Accuracy on {} vali file is {}".format("Fold5", metrics.accuracy_score(y_vali, predictions)))
# apply the model on test file and print
print("For each qid in Fold5 test file, sort the documents:")
sorted_docid_print()

## Part 2.2: NDCG (20 points)

Based on your prediction file (results could be ranked by scores in the prediction file) and ground-truth (i.e., 0,1,2) in the test file, calculate NDCG for each query. Report average NDCG for all queries in the five-fold cross validation.

For NDCG, please bulid your own function rather then using any package.

In [29]:
# your code here
def NDCG():
    c = clf.predict_log_proba (x_test)
    score_list = []
    for i in range(len(c)):
        score_list.append(c[i][1])
    qid_docid_score_dict = {}
    for i in range(len(c)):
        qid_docid_list = [docid_test[i], score_list[i], rel_test[i]]
        if qid_test[i] not in qid_docid_score_dict:
            qid_docid_score_dict[qid_test[i]] = [qid_docid_list]
        else:
            qid_docid_score_dict[qid_test[i]].append(qid_docid_list)
    sum_NDCG = 0
    for key, value in qid_docid_score_dict.items():
        value.sort(key=lambda x: x[1], reverse=True)
    #     print(key)
    #     for i in range(len(value)):
    #         print(value[i][0], '\t', value[i][1], '\t', value[i][2])
        DCG = 0
        if len(value) >= 10:
            for j in range(10):
                DCG += value[j][2]/np.log2(j+2)
        else:
            for j in range(len(value)):
                DCG += value[j][2]/np.log2(j+2)
        # IDCG
        value.sort(key=lambda x: x[2], reverse=True)
        IDCG = 0
        for j in range(len(value)):
                IDCG += value[j][2]/np.log2(j+2)
        # NDCG
        if IDCG == 0:
            NDCG = 0
        else:
            NDCG = DCG/IDCG
        sum_NDCG += NDCG
        
    average_NDCG = sum_NDCG/len(qid_docid_score_dict)
    return average_NDCG

In [30]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold1\\train.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold1\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
Fold1_average_NDCG = NDCG()
print("Fold1_average_NDCG: ", Fold1_average_NDCG)

Fold1_average_NDCG:  0.431112388577871


In [31]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold2\\train.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold2\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
Fold2_average_NDCG = NDCG()
print("Fold2_average_NDCG: ", Fold2_average_NDCG)

Fold2_average_NDCG:  0.433529582537733


In [32]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold3\\train.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold3\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
Fold3_average_NDCG = NDCG()
print("Fold3_average_NDCG: ", Fold3_average_NDCG)

Fold3_average_NDCG:  0.4538636734023412


In [33]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold4\\train.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold4\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
Fold4_average_NDCG = NDCG()
print("Fold4_average_NDCG: ", Fold4_average_NDCG)

Fold4_average_NDCG:  0.5022472279652941


In [34]:
x_train, y_train = createTrainDataSet("MQ2008\\MQ2008\\Fold5\\train.txt")
x_test, y_test, rel_test, qid_test, docid_test = createTestDataSet("MQ2008\\MQ2008\\Fold5\\test.txt")

clf = SVC(gamma='auto', probability=True)
clf.fit(x_train, y_train)
Fold5_average_NDCG = NDCG()
print("Fold5_average_NDCG: ", Fold5_average_NDCG)

Fold5_average_NDCG:  0.5091206325695353


In [35]:
sum_NDCG = Fold1_average_NDCG + Fold2_average_NDCG + Fold3_average_NDCG + Fold4_average_NDCG + Fold5_average_NDCG
NDCG = sum_NDCG / 5
print("NDCG in the whole dataset: ", NDCG)

NDCG in the whole dataset:  0.4659747010105549


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*